# Import the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
%matplotlib inline

# Connecting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import files

In [ ]:
folder_path = '/content/drive/MyDrive/EC439/CSV Data'
all_files = os.listdir(folder_path)

# Creating and Cleaning DataFrames

## Cash Flow DataFrame

In [ ]:
cf_files = [f for f in all_files if f.endswith('.csv') and 'cashflow' in f]

cf_dfs = {}
for file in cf_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if 'Period End Date' in line:
                skiprows = i
                break
        else:
            skiprows = 0  # If 'Period End Date' not found, don't skip any rows
    cf_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None)

    # Transpose
    cf_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None).transpose()
    # Shift first row to header
    cf_dfs[file].columns = cf_dfs[file].iloc[0]
    cf_dfs[file] = cf_dfs[file][1:]

In [ ]:
print(cf_dfs.keys())
print(len(cf_dfs.keys()))

dict_keys(['AKR.cashflow (STARK).xls.csv', 'ACE.cashflow (EA).csv', 'BBGI.cashflow (EA).csv', 'BCPG.cashflow (EA).csv', 'CKP.cashflow (EA).csv', 'CPT.cashflow (STARK).csv', 'ETC.cashflow (EA).csv', 'EA.cashflow.csv', 'SCI.cashflow (STARK).csv', 'LNE.cashflow (STARK).csv', 'SSP.cashflow (EA).csv', 'SPCG.cashflow (EA).csv', 'SUPER.cashflow (EA).csv', 'TMIm.cashflow (STARK).csv', 'TGE.cashflow (EA).csv', 'TPIPP.cashflow (EA).csv', 'TRTm.cashflow (STARK).csv', 'STARK.cashflow.csv'])
18


In [ ]:
# Get the columns for the first DataFrame as a starting point
all_columns = set(cf_dfs[list(cf_dfs.keys())[0]].columns)

# Find common columns (intersection)
common_columns = all_columns.intersection(*[set(df.columns) for df in cf_dfs.values()])

# Find different columns (unique to each DataFrame)
different_columns = {}
for key, df in cf_dfs.items():
    unique_cols = set(df.columns) - common_columns
    if unique_cols:  # Add only if there are unique columns
        different_columns[key] = list(unique_cols)

# Print the results
print("Common Columns:", list(common_columns))
print("\nDifferent Columns (Unique to each DataFrame):")
for key, cols in different_columns.items():
    print(f"{key}: {cols}")

# Count common and different columns
num_common_cols = len(common_columns)
num_different_cols = sum(len(cols) for cols in different_columns.values())
print("-" * 20)
# Print the results
print(f"Number of Common Columns: {num_common_cols}")
print(f"Number of Different Columns: {num_different_cols}")

Common Columns: [nan, 'Other Assets', 'Capital Expenditures', 'Cash Taxes Paid', 'Period End Date', 'Cash Flow-Operating Activities (THB Thousands)', 'Cash Interest Paid', 'Net Change in Cash', 'Cash from Financing Activities', 'Net Cash - Beginning Balance', 'Non-Cash Items', 'Cash from Operating Activities', 'Free Cash Flow', 'Issuance (Retirement) of Debt, Net', 'Purchase of Fixed Assets', 'Cash Flow-Investing Activities (THB Thousands)', 'Other Liabilities', 'Cash Flow-Financing Activities (THB Thousands)', 'Other Operating Cash Flow', 'Lease liability Reduced, Supplemental', 'Net Income/Starting Line', 'Other Investing Cash Flow', 'Other Financing Cash Flow', 'Long Term Debt, Net', 'Accounts Receivable', 'Financing Cash Flow Items', 'Unusual Items', 'Depreciation', 'Sale of Fixed Assets', 'Net Cash - Ending Balance', 'Changes in Working Capital', 'Depreciation/Depletion', 'Cash from Investing Activities', 'Sale/Maturity of Investment', 'Long Term Debt Reduction', 'Other Non-Cash I

In [ ]:
# Drop different columns in each DataFrame and update cf_dfs
for key in cf_dfs:
    cf_dfs[key] = cf_dfs[key][list(common_columns)]

In [ ]:
for key in cf_dfs:
    cf_dfs[key] = cf_dfs[key].replace('--', 0)
    cf_dfs[key] = cf_dfs[key].replace([' ', '  ', np.nan, None], np.nan)

<ipython-input-8-d725845d4b7b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cf_dfs[key] = cf_dfs[key].replace('--', 0)


In [ ]:
for key, df in cf_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.cashflow (STARK).xls.csv
Column 'nan' has 80 NaN values.
Column 'nan' has 80 NaN values.
Column 'nan' has 80 NaN values.
Column 'nan' has 80 NaN values.
Column 'nan' has 80 NaN values.
Column 'nan' has 80 NaN values.
Column 'Cash Flow-Operating Activities (THB Thousands)' has 80 NaN values.
Column 'Cash Flow-Investing Activities (THB Thousands)' has 80 NaN values.
Column 'Cash Flow-Financing Activities (THB Thousands)' has 80 NaN values.
--------------------
File: ACE.cashflow (EA).csv
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'Cash Flow-Operating Activities (THB Thousands)' has 26 NaN values.
Column 'Cash Flow-Investing Activities (THB Thousands)' has 26 NaN values.
Column 'Cash Flow-Financing Activities (THB Thousands)' has 26 NaN values.
--------------------
File: BBGI.cashflow (EA).csv
Column 'nan' has 14 NaN values.


In [ ]:
for key, df in cf_dfs.items():
    columns_to_drop = ['Cash Flow-Operating Activities (THB Thousands)',
                      'Cash Flow-Investing Activities (THB Thousands)',
                      'Cash Flow-Financing Activities (THB Thousands)']

    # Drop columns if they exist in the DataFrame
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    df = df.dropna(axis=1, how='all')

    # Update the DataFrame in cf_dfs
    cf_dfs[key] = df

In [ ]:
for key, df in cf_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.cashflow (STARK).xls.csv
--------------------
File: ACE.cashflow (EA).csv
--------------------
File: BBGI.cashflow (EA).csv
--------------------
File: BCPG.cashflow (EA).csv
--------------------
File: CKP.cashflow (EA).csv
--------------------
File: CPT.cashflow (STARK).csv
--------------------
File: ETC.cashflow (EA).csv
--------------------
File: EA.cashflow.csv
--------------------
File: SCI.cashflow (STARK).csv
--------------------
File: LNE.cashflow (STARK).csv
--------------------
File: SSP.cashflow (EA).csv
--------------------
File: SPCG.cashflow (EA).csv
--------------------
File: SUPER.cashflow (EA).csv
--------------------
File: TMIm.cashflow (STARK).csv
--------------------
File: TGE.cashflow (EA).csv
--------------------
File: TPIPP.cashflow (EA).csv
--------------------
File: TRTm.cashflow (STARK).csv
--------------------
File: STARK.cashflow.csv
--------------------


In [ ]:
for key in cf_dfs:
    # Get a list of columns, with 'Period End Date' moved to the front
    cols = ['Period End Date'] + [col for col in cf_dfs[key].columns if col != 'Period End Date']

    # Reorder the DataFrame columns
    cf_dfs[key] = cf_dfs[key][cols]

In [ ]:
# Get the column names from the first DataFrame as the reference order
reference_columns = cf_dfs[list(cf_dfs.keys())[0]].columns

# Reorder columns in all DataFrames to match the reference order
for key in cf_dfs:
    cf_dfs[key] = cf_dfs[key][reference_columns]

In [ ]:
for key in cf_dfs:
    # Extract company name using regular expression
    company_name = re.search(r'(.*)\.cashflow', key).group(1)

    # Add 'Company name' column
    cf_dfs[key]['Company name'] = company_name

In [ ]:
for key in cf_dfs:
    # Add 'Fraud' column and initialize to 0
    cf_dfs[key]['Fraud'] = 0

    # Set 'Fraud' to 1 for specific companies
    company_name = cf_dfs[key]['Company name'].iloc[0]  # Get company name
    if company_name in ['EA', 'STARK']:
        cf_dfs[key]['Fraud'] = 1

In [ ]:
def compare_columns_in_dict(dfs_dict):
  results = {}
  first_df_columns = sorted(list(dfs_dict.values())[0].columns)  # Get columns of the first DataFrame

  for key, df in dfs_dict.items():
    current_df_columns = sorted(df.columns)
    if current_df_columns == first_df_columns:
      results[key] = "Yes"
    else:
      results[key] = "No"

  return results

# Example usage:
column_comparison_results = compare_columns_in_dict(cf_dfs)

for file, result in column_comparison_results.items():
  print(f"File: {file}, Columns Same: {result}")

File: AKR.cashflow (STARK).xls.csv, Columns Same: Yes
File: ACE.cashflow (EA).csv, Columns Same: Yes
File: BBGI.cashflow (EA).csv, Columns Same: Yes
File: BCPG.cashflow (EA).csv, Columns Same: Yes
File: CKP.cashflow (EA).csv, Columns Same: Yes
File: CPT.cashflow (STARK).csv, Columns Same: Yes
File: ETC.cashflow (EA).csv, Columns Same: Yes
File: EA.cashflow.csv, Columns Same: Yes
File: SCI.cashflow (STARK).csv, Columns Same: Yes
File: LNE.cashflow (STARK).csv, Columns Same: Yes
File: SSP.cashflow (EA).csv, Columns Same: Yes
File: SPCG.cashflow (EA).csv, Columns Same: Yes
File: SUPER.cashflow (EA).csv, Columns Same: Yes
File: TMIm.cashflow (STARK).csv, Columns Same: Yes
File: TGE.cashflow (EA).csv, Columns Same: Yes
File: TPIPP.cashflow (EA).csv, Columns Same: Yes
File: TRTm.cashflow (STARK).csv, Columns Same: Yes
File: STARK.cashflow.csv, Columns Same: Yes


In [ ]:
cf_dfs_merged = pd.concat(cf_dfs.values(), ignore_index=True)
print(len(cf_dfs_merged.columns.to_list()))
cf_dfs_merged

36


,Period End Date,Other Assets,Capital Expenditures,Cash Taxes Paid,Cash Interest Paid,Net Change in Cash,Cash from Financing Activities,Net Cash - Beginning Balance,Non-Cash Items,Cash from Operating Activities,...,Changes in Working Capital,Depreciation/Depletion,Cash from Investing Activities,Sale/Maturity of Investment,Long Term Debt Reduction,Other Non-Cash Items,Net Changes in Working Capital,"Other Investing Cash Flow Items, Total",Company name,Fraud
0,30-Jun-2004,"(7,347)","(3,087)","2,720","2,116","40,499","(11,402)","19,400","(7,705)","54,229",...,"(55,841)","56,369","(2,900)",0,(626),"(6,790)","(55,841)",187,AKR,0
1,31-Dec-2004,"(20,705)","(78,341)","5,126","26,881","14,550","50,608","19,400","(1,501,984)","47,939",...,"(98,420)","113,428","(77,913)",25,"(643,253)","(11,762)","(98,420)",428,AKR,0
2,31-Mar-2005,"(26,572)","(3,204)","1,230","3,894","(10,854)","85,686","33,950",(443),"(87,610)",...,"(95,091)","5,404","(8,946)",0,"(167,628)",(532),"(95,091)","(5,742)",AKR,0
3,30-Jun-2005,(879),"(14,055)","2,116","7,217","4,345","202,012","33,950",(959),"(177,886)",...,"(261,066)","15,741","(19,797)",0,"(182,932)","(1,048)","(261,066)","(5,742)",AKR,0
4,30-Sep-2005,"(12,774)","(27,921)","5,117","14,689","12,409","155,207","33,950","9,401","(109,151)",...,"(273,379)","26,213","(33,663)",0,"(195,323)","9,365","(273,379)","(5,742)",AKR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,31-Dec-2021,"(98,648)","(181,675)","412,408","852,807","(126,552)","1,560,915","1,156,030","1,362,045","(1,225,481)",...,"(6,547,110)","423,133","(406,021)",0,"(1,575,177)","1,774,757","(5,270,780)","(224,346)",STARK,1
901,31-Mar-2022,"(715,951)","(89,853)","354,484","242,535","(382,466)","619,340","1,029,478","202,093","(852,473)",...,"(1,971,432)","108,089","(141,138)",0,"(559,661)","235,217","(1,376,685)","(51,286)",STARK,1
902,30-Jun-2022,"21,459","(184,598)","618,700","687,183","(303,596)","2,040,745","1,029,478","562,149","(2,044,095)",...,"(4,623,644)","249,792","(177,955)",0,"(3,201,573)","676,189","(3,316,285)","6,643",STARK,1
903,30-Sep-2022,"77,613","(182,167)","710,433","859,418","(575,997)","1,350,831","1,029,478","1,548,418","(1,798,569)",...,"(6,645,239)","331,453","(173,691)",0,"(3,599,175)","1,090,359","(5,076,394)","8,476",STARK,1


In [ ]:
cf_dfs_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Period End Date                         905 non-null    object
 1   Other Assets                            905 non-null    object
 2   Capital Expenditures                    905 non-null    object
 3   Cash Taxes Paid                         905 non-null    object
 4   Cash Interest Paid                      905 non-null    object
 5   Net Change in Cash                      905 non-null    object
 6   Cash from Financing Activities          905 non-null    object
 7   Net Cash - Beginning Balance            905 non-null    object
 8   Non-Cash Items                          905 non-null    object
 9   Cash from Operating Activities          905 non-null    object
 10  Free Cash Flow                          905 non-null    object
 11  Issuan

In [ ]:
cf_df = cf_dfs_merged

In [ ]:
def acc_tran(num):
  if type(num) != str:
    return num

  # print(num)
  # print(type(num))
  num = num.replace(",", "")
  if (num[0] == "(" and num[-1] == ")"):
      num = num[1:-1]
      num = float(num) * (-1)
  else:
      num = float(num)

  return num

In [ ]:
column_names = cf_df.columns
blacklist = ["Period End Date","Company name","Fraud"]
for col in column_names:
    if col not in blacklist:
      cf_df[col] = cf_df[col].apply(acc_tran)

print(cf_df)

0   Period End Date  Other Assets  Capital Expenditures  Cash Taxes Paid  \
0      30-Jun-2004        -7347.0               -3087.0           2720.0   
1      31-Dec-2004       -20705.0              -78341.0           5126.0   
2      31-Mar-2005       -26572.0               -3204.0           1230.0   
3      30-Jun-2005         -879.0              -14055.0           2116.0   
4      30-Sep-2005       -12774.0              -27921.0           5117.0   
..              ...           ...                   ...              ...   
900    31-Dec-2021       -98648.0             -181675.0         412408.0   
901    31-Mar-2022      -715951.0              -89853.0         354484.0   
902    30-Jun-2022        21459.0             -184598.0         618700.0   
903    30-Sep-2022        77613.0             -182167.0         710433.0   
904    31-Dec-2022       194504.0             -152942.0         688863.0   

0    Cash Interest Paid  Net Change in Cash  Cash from Financing Activities  \
0       

In [ ]:
cf_df.dtypes

,0
0,
Period End Date,object
Other Assets,float64
Capital Expenditures,float64
Cash Taxes Paid,float64
Cash Interest Paid,float64
Net Change in Cash,float64
Cash from Financing Activities,float64
Net Cash - Beginning Balance,float64
Non-Cash Items,float64


In [ ]:
cf_df.to_csv('cf_df.csv', index=False)

## Price History DataFrame

In [ ]:
ph_files = [f for f in all_files if f.endswith('.csv') and 'Price History' in f]

ph_dfs = {}
for file in ph_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if 'Exchange Date' in line:
                skiprows = i
                break
        else:
            skiprows = 0  # If 'Exchange Date' not found, don't skip any rows
    ph_dfs[file] = pd.read_csv(file_path, skiprows=skiprows)

In [ ]:
print(len(ph_dfs.keys()))
ph_dfs["AKR.Price History (STARK).csv"].head()

18


,Exchange Date,Close,Net,%Chg,Open,Low,High,Volume,Turnover - THB,Flow,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,27-ก.ย.-2024,0.96,0.01,+1.05%,0.95,0.95,0.98,"3,590,000","3,450,017.00","12,750,049,352.00",NaN,NaN,NaN,NaN,NaN,NaN
1,20-ก.ย.-2024,0.95,0.01,+1.06%,0.95,0.93,0.98,"11,836,300","11,337,320.00","12,746,599,335.00",NaN,NaN,NaN,NaN,NaN,NaN
2,13-ก.ย.-2024,0.94,0.01,+1.08%,0.94,0.91,0.97,"15,850,800","14,971,829.00","12,735,262,015.00",NaN,NaN,NaN,NaN,NaN,NaN
3,06-ก.ย.-2024,0.93,0.03,+3.33%,0.91,0.88,0.93,"10,390,600","9,478,282.00","12,720,290,186.00",NaN,NaN,NaN,NaN,NaN,NaN
4,30-ส.ค.-2024,0.90,0.02,+2.27%,0.89,0.88,0.92,"8,502,700","7,661,700.00","12,710,811,904.00",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for key in ph_dfs:
    ph_dfs[key] = ph_dfs[key][['Exchange Date', 'Close']]

ph_dfs["AKR.Price History (STARK).csv"].head()

,Exchange Date,Close
0,27-ก.ย.-2024,0.96
1,20-ก.ย.-2024,0.95
2,13-ก.ย.-2024,0.94
3,06-ก.ย.-2024,0.93
4,30-ส.ค.-2024,0.90


In [ ]:
for key, df in ph_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.Price History (STARK).csv
--------------------
File: ACE.Price History (EA).csv
--------------------
File: BBGI.Price History (EA).csv
--------------------
File: BCPG.Price History (EA).csv
--------------------
File: CKP.Price History (EA).csv
--------------------
File: CPT.Price History (STARK).csv
--------------------
File: EA.Price History.csv
--------------------
File: ETC.Price History (EA).csv
--------------------
File: LNE.Price History (STARK).csv
--------------------
File: SCI.Price History (STARK).csv
--------------------
File: SPCG.Price History (EA).csv
--------------------
File: SSP.Price History (EA).csv
--------------------
File: STARK.Price History.csv
Column 'Close' has 4 NaN values.
--------------------
File: SUPER.Price History (EA).csv
--------------------
File: TGE.Price History (EA).csv
--------------------
File: TMIm.Price History (STARK).csv
--------------------
File: TPIPP.Price History (EA).csv
--------------------
File: TRTm.Price History (STARK).cs

In [ ]:
for key in ph_dfs:
    ph_dfs[key]['Close'] = pd.to_numeric(ph_dfs[key]['Close'], errors='coerce').fillna(0)

In [ ]:
for key, df in ph_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.Price History (STARK).csv
--------------------
File: ACE.Price History (EA).csv
--------------------
File: BBGI.Price History (EA).csv
--------------------
File: BCPG.Price History (EA).csv
--------------------
File: CKP.Price History (EA).csv
--------------------
File: CPT.Price History (STARK).csv
--------------------
File: EA.Price History.csv
--------------------
File: ETC.Price History (EA).csv
--------------------
File: LNE.Price History (STARK).csv
--------------------
File: SCI.Price History (STARK).csv
--------------------
File: SPCG.Price History (EA).csv
--------------------
File: SSP.Price History (EA).csv
--------------------
File: STARK.Price History.csv
--------------------
File: SUPER.Price History (EA).csv
--------------------
File: TGE.Price History (EA).csv
--------------------
File: TMIm.Price History (STARK).csv
--------------------
File: TPIPP.Price History (EA).csv
--------------------
File: TRTm.Price History (STARK).csv
--------------------


In [ ]:
for key in ph_dfs:
    # Extract company name using regular expression
    company_name = re.search(r'(.*)\.Price History', key).group(1)

    # Add 'Company name' column
    ph_dfs[key]['Company name'] = company_name

In [ ]:
for key in ph_dfs:
    # Add 'Fraud' column and initialize to 0
    ph_dfs[key]['Fraud'] = 0

    # Set 'Fraud' to 1 for specific companies
    company_name = ph_dfs[key]['Company name'].iloc[0]  # Get company name
    if company_name in ['EA', 'STARK']:
        ph_dfs[key]['Fraud'] = 1

In [ ]:
ph_dfs['STARK.Price History.csv'].head(10)

,Exchange Date,Close,Company name,Fraud
0,06-ก.ย.-2024,0.01,STARK,1
1,30-ส.ค.-2024,0.01,STARK,1
2,23-ส.ค.-2024,0.01,STARK,1
3,28-ก.ค.-2023,0.00,STARK,1
4,21-ก.ค.-2023,0.00,STARK,1
5,14-ก.ค.-2023,0.00,STARK,1
6,07-ก.ค.-2023,0.00,STARK,1
7,30-มิ.ย.-2023,0.02,STARK,1
8,23-มิ.ย.-2023,0.02,STARK,1
9,16-มิ.ย.-2023,0.08,STARK,1


In [ ]:
ph_dfs_merged = pd.concat(ph_dfs.values(), ignore_index=True)
ph_df = ph_dfs_merged
ph_df

,Exchange Date,Close,Company name,Fraud
0,27-ก.ย.-2024,0.96,AKR,0
1,20-ก.ย.-2024,0.95,AKR,0
2,13-ก.ย.-2024,0.94,AKR,0
3,06-ก.ย.-2024,0.93,AKR,0
4,30-ส.ค.-2024,0.90,AKR,0
...,...,...,...,...
7343,24-ต.ค.-2014,5.90,TRTm,0
7344,17-ต.ค.-2014,5.80,TRTm,0
7345,10-ต.ค.-2014,6.20,TRTm,0
7346,03-ต.ค.-2014,5.70,TRTm,0


In [ ]:
ph_df.dtypes

,0
Exchange Date,object
Close,float64
Company name,object
Fraud,int64


In [ ]:
ph_df.to_csv('ph_df.csv', index=False)

## Balancesheet DataFrame

In [ ]:
bs_files = [f for f in all_files if f.endswith('.csv') and 'balancesheet' in f]

bs_dfs = {}
# bs_dfs_test = {}
for file in bs_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
          # print(i,"-----",line)
          if 'Period End Date' in line:
                skiprows = i
                break
        else:
            skiprows = 0  # If 'Period End Date' not found, don't skip any rows
    bs_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None)

    # Transpose
    bs_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None).transpose()
    # Shift first row to header
    bs_dfs[file].columns = bs_dfs[file].iloc[0]
    bs_dfs[file] = bs_dfs[file][1:]

In [ ]:
print(len(bs_dfs.keys()))
bs_dfs['AKR.balancesheet (STARK).xls.csv'].head(10)

18


,Period End Date,Assets (THB Thousands),Cash and Short Term Investments,Cash,Cash & Equivalents,Short Term Investments,"Accounts Receivable - Trade, Net","Accounts Receivable - Trade, Gross",Provision for Doubtful Accounts,"Total Receivables, Net",...,Capital Lease Payments Due in 4-5 Years,Cap. Lease Pymts. Due in Year 6 & Beyond,Total Funded Status,Unfunded Plan Obligations,Period End Assumptions,Discount Rate - Domestic,Compensation Rate - Domestic,Net Assets Recognized on Balance Sheet,Accrued Liabilities - Domestic,Total Plan Obligations
1,31-Dec-2004,NaN,"33,950",--,"33,950",--,"389,706","425,090","(35,384)","394,115",...,--,--,--,--,--,--,--,--,--,--
2,30-Jun-2005,NaN,"38,295",--,"38,295",--,"407,976","418,756","(10,780)","422,799",...,--,--,--,--,--,--,--,--,--,--
3,31-Dec-2005,NaN,"45,014",--,"45,014",--,"316,608","326,237","(9,629)","337,608",...,--,--,--,--,--,--,--,--,--,--
4,31-Mar-2006,NaN,"87,349",--,"87,349",--,"280,428","291,767","(11,339)","292,276",...,--,--,--,--,--,--,--,--,--,--
5,30-Jun-2006,NaN,"46,049",--,"46,049",--,"378,248","389,935","(11,687)","390,979",...,--,--,--,--,--,--,--,--,--,--
6,30-Sep-2006,NaN,"151,603",--,"151,603",--,"404,091","415,704","(11,613)","430,778",...,--,--,--,--,--,--,--,--,--,--
7,31-Dec-2006,NaN,"103,532",--,"103,223",309,"362,982","375,714","(12,732)","405,173",...,--,--,--,--,--,--,--,--,--,--
8,31-Mar-2007,NaN,"226,105",--,"195,671","30,434","238,902","253,490","(14,588)","303,729",...,--,--,--,--,--,--,--,--,--,--
9,30-Jun-2007,NaN,"49,881",--,"49,536",345,"224,210","241,531","(17,321)","294,973",...,--,--,--,--,--,--,--,--,--,--
10,30-Sep-2007,NaN,"77,057",--,"76,689",368,"455,879","473,669","(17,790)","549,293",...,--,--,--,--,--,--,--,--,--,--


In [ ]:
# Get the columns for the first DataFrame as a starting point
all_columns = set(bs_dfs[list(bs_dfs.keys())[0]].columns)

# Find common columns (intersection)
common_columns = all_columns.intersection(*[set(df.columns) for df in bs_dfs.values()])

# Find different columns (unique to each DataFrame)
different_columns = {}
for key, df in bs_dfs.items():
    unique_cols = set(df.columns) - common_columns
    if unique_cols:  # Add only if there are unique columns
        different_columns[key] = list(unique_cols)

# Print the results
print("Common Columns:", list(common_columns))
print("\nDifferent Columns (Unique to each DataFrame):")
for key, cols in different_columns.items():
    print(f"{key}: {cols}")

# Count common and different columns
num_common_cols = len(common_columns)
num_different_cols = sum(len(cols) for cols in different_columns.values())
print("-" * 20)
# Print the results
print(f"Number of Common Columns: {num_common_cols}")
print(f"Number of Different Columns: {num_different_cols}")

Common Columns: ['Total Current Liabilities', nan, 'Minority Interest', 'Minority Interest - Non Redeemable', 'Supplemental (THB Thousands)', 'Period End Date', 'Total Current Assets less Inventory', 'Accounts Receivable - Trade, Gross', 'Common Stock', 'Total Common Shares Outstanding', 'Receivables - Other', 'Liabilities (THB Thousands)', 'Defered Income Tax - Long Term Asset', 'Total Debt', 'Total Long Term Debt', 'Notes Payable/Short Term Debt', 'Other Property/Plant/Equipment - Net', 'Shares Outs - Common Stock Primary Issue', 'Other Long Term Assets, Total', 'Curr. Port. of LT Capital Leases, Suppl.', "Total Liabilities & Shareholders' Equity", 'Property/Plant/Equipment, Total - Net', 'Other Liabilities, Total', 'Total Receivables, Net', 'Shareholders Equity (THB Thousands)', 'Assets (THB Thousands)', 'Other Payables', 'Capital Lease Obligations', 'Total Equity', 'Other Long Term Assets', 'Total Current Assets', 'Long Term Debt', 'Cash & Equivalents', 'Total Liabilities', 'Accoun

In [ ]:
# Drop different columns in each DataFrame and update bs_dfs
for key in bs_dfs:
    bs_dfs[key] = bs_dfs[key][list(common_columns)]

In [ ]:
for key in bs_dfs:
    bs_dfs[key] = bs_dfs[key].replace('--', 0)
    bs_dfs[key] = bs_dfs[key].replace([' ', '  ', np.nan, None], np.nan)

<ipython-input-40-dec1e73c6678>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bs_dfs[key] = bs_dfs[key].replace('--', 0)
<ipython-input-40-dec1e73c6678>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bs_dfs[key] = bs_dfs[key].replace([' ', '  ', np.nan, None], np.nan)


In [ ]:
for key, df in bs_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.balancesheet (STARK).xls.csv
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'Supplemental (THB Thousands)' has 77 NaN values.
Column 'Liabilities (THB Thousands)' has 77 NaN values.
Column 'Shareholders Equity (THB Thousands)' has 77 NaN values.
Column 'Assets (THB Thousands)' has 77 NaN values.
--------------------
File: ACE.balancesheet (EA).csv
Column 'nan' has 24 NaN values.
Column 'nan' has 24 NaN values.
Column 'nan' has 24 NaN values.
Column 'nan' has 24 NaN values.
Column 'nan' has 24 NaN values.
Column 'nan' has 24 NaN values.
Column 'Supplemental (THB Thousands)' has 24 NaN values.
Column 'Liabilities (THB Thousands)' has 24 NaN values.
Column 'Shareholders Equity (THB Thousands)' has 24 NaN values.
Column 'Assets (THB Thousands)' has 24 NaN values.
--------------------
File: BBGI.balancesheet (EA).csv
Column 'nan' h

In [ ]:
for key, df in bs_dfs.items():
    columns_to_drop = ['Assets (THB Thousands)',
                      'Liabilities (THB Thousands)',
                      'Shareholders Equity (THB Thousands)',
                       'Supplemental (THB Thousands)']

    # Drop columns if they exist in the DataFrame
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    df = df.dropna(axis=1, how = 'all')

    # Update the DataFrame in bs_dfs
    bs_dfs[key] = df

In [ ]:
for key, df in bs_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.balancesheet (STARK).xls.csv
--------------------
File: ACE.balancesheet (EA).csv
--------------------
File: BBGI.balancesheet (EA).csv
--------------------
File: BCPG.balancesheet (EA).csv
--------------------
File: CKP.balancesheet (EA).csv
--------------------
File: CPT.balancesheet (STARK).csv
--------------------
File: ETC.balancesheet (EA).csv
--------------------
File: EA.balancesheet.csv
--------------------
File: SCI.balancesheet (STARK).csv
--------------------
File: LNE.balancesheet (STARK).csv
--------------------
File: SSP.balancesheet (EA).csv
--------------------
File: SPCG.balancesheet (EA).csv
--------------------
File: SUPER.balancesheet (EA).csv
--------------------
File: STARK.balancesheet.csv
--------------------
File: TMIm.balancesheet (STARK).csv
--------------------
File: TGE.balancesheet (EA).csv
--------------------
File: TRTm.balancesheet (STARK).csv
--------------------
File: TPIPP.balancesheet (EA).csv
--------------------


In [ ]:
for key in bs_dfs:
    # Get a list of columns, with 'Period End Date' moved to the front
    cols = ['Period End Date'] + [col for col in bs_dfs[key].columns if col != 'Period End Date']

    # Reorder the DataFrame columns
    bs_dfs[key] = bs_dfs[key][cols]

In [ ]:
# Get the column names from the first DataFrame as the reference order
reference_columns = bs_dfs[list(bs_dfs.keys())[0]].columns

# Reorder columns in all DataFrames to match the reference order
for key in bs_dfs:
    bs_dfs[key] = bs_dfs[key][reference_columns]

In [ ]:
for key in bs_dfs:
    # Extract company name using regular expression
    company_name = re.search(r'(.*)\.balancesheet', key).group(1)

    # Add 'Company name' column
    bs_dfs[key]['Company name'] = company_name

bs_dfs["AKR.balancesheet (STARK).xls.csv"]

,Period End Date,Total Current Liabilities,Minority Interest,Minority Interest - Non Redeemable,Total Current Assets less Inventory,"Accounts Receivable - Trade, Gross",Common Stock,Total Common Shares Outstanding,Receivables - Other,Defered Income Tax - Long Term Asset,...,Current Port. of LT Debt/Capital Leases,"Tangible Book Value, Common Equity",Long Term Investments,Treas Shares - Common Stock Prmry Issue,Cash and Short Term Investments,"Other Current liabilities, Total",Additional Paid-In Capital,"Common Stock, Total",Retained Earnings (Accumulated Deficit),Company name
1,31-Dec-2004,"821,500","(179,534)","(179,534)","447,718","425,090","508,174","1,002,140","4,409",0,...,"107,124","213,202",0,0,"33,950","190,948","107,500","508,174","(167,701)",AKR
2,30-Jun-2005,"626,215",0,0,"492,350","418,756","608,174","1,002,140","14,823",0,...,"58,068","513,794",0,0,"38,295","60,269","207,500","608,174","(96,783)",AKR
3,31-Dec-2005,"703,892",0,0,"410,495","326,237","608,174","1,002,140","21,000",0,...,"61,400","620,668",0,0,"45,014","77,238","4,924","608,174","7,570",AKR
4,31-Mar-2006,"785,650",0,0,"417,965","291,767","608,174","1,002,140","11,848",0,...,"61,430","624,519",0,0,"87,349","102,648","4,924","608,174","11,421",AKR
5,30-Jun-2006,"764,503",0,0,"476,753","389,935","608,174","1,002,140","12,731",0,...,"61,460","669,173",0,0,"46,049","98,828","4,924","608,174","56,075",AKR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,30-Jun-2023,"685,937",288,288,"511,883","391,843","1,177,226","1,471,533",0,"66,365",...,"32,875","1,345,452","21,282",0,"99,843","99,704","25,649","1,177,226","165,533",AKR
74,30-Sep-2023,"684,015",283,283,"547,488","415,747","1,177,226","1,471,533",0,"65,544",...,"38,222","1,402,985","21,282",0,"103,572","74,544","25,649","1,177,226","222,418",AKR
75,31-Dec-2023,"604,710",474,474,"631,191","357,468","1,177,226","1,471,533",0,"60,670",...,"35,422","1,472,100","22,075",0,"261,374","90,394","25,649","1,177,226","291,907",AKR
76,31-Mar-2024,"670,841","1,468","1,468","586,219","363,996","1,177,226","1,471,533",0,"58,890",...,"30,830","1,510,652","22,075",0,"207,202","97,226","25,649","1,177,226","329,873",AKR


In [ ]:
for key in bs_dfs:
    # Add 'Fraud' column and initialize to 0
    bs_dfs[key]['Fraud'] = 0

    # Set 'Fraud' to 1 for specific companies
    company_name = bs_dfs[key]['Company name'].iloc[0]  # Get company name
    if company_name in ['EA', 'STARK']:
        bs_dfs[key]['Fraud'] = 1

bs_dfs["AKR.balancesheet (STARK).xls.csv"]

,Period End Date,Total Current Liabilities,Minority Interest,Minority Interest - Non Redeemable,Total Current Assets less Inventory,"Accounts Receivable - Trade, Gross",Common Stock,Total Common Shares Outstanding,Receivables - Other,Defered Income Tax - Long Term Asset,...,"Tangible Book Value, Common Equity",Long Term Investments,Treas Shares - Common Stock Prmry Issue,Cash and Short Term Investments,"Other Current liabilities, Total",Additional Paid-In Capital,"Common Stock, Total",Retained Earnings (Accumulated Deficit),Company name,Fraud
1,31-Dec-2004,"821,500","(179,534)","(179,534)","447,718","425,090","508,174","1,002,140","4,409",0,...,"213,202",0,0,"33,950","190,948","107,500","508,174","(167,701)",AKR,0
2,30-Jun-2005,"626,215",0,0,"492,350","418,756","608,174","1,002,140","14,823",0,...,"513,794",0,0,"38,295","60,269","207,500","608,174","(96,783)",AKR,0
3,31-Dec-2005,"703,892",0,0,"410,495","326,237","608,174","1,002,140","21,000",0,...,"620,668",0,0,"45,014","77,238","4,924","608,174","7,570",AKR,0
4,31-Mar-2006,"785,650",0,0,"417,965","291,767","608,174","1,002,140","11,848",0,...,"624,519",0,0,"87,349","102,648","4,924","608,174","11,421",AKR,0
5,30-Jun-2006,"764,503",0,0,"476,753","389,935","608,174","1,002,140","12,731",0,...,"669,173",0,0,"46,049","98,828","4,924","608,174","56,075",AKR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,30-Jun-2023,"685,937",288,288,"511,883","391,843","1,177,226","1,471,533",0,"66,365",...,"1,345,452","21,282",0,"99,843","99,704","25,649","1,177,226","165,533",AKR,0
74,30-Sep-2023,"684,015",283,283,"547,488","415,747","1,177,226","1,471,533",0,"65,544",...,"1,402,985","21,282",0,"103,572","74,544","25,649","1,177,226","222,418",AKR,0
75,31-Dec-2023,"604,710",474,474,"631,191","357,468","1,177,226","1,471,533",0,"60,670",...,"1,472,100","22,075",0,"261,374","90,394","25,649","1,177,226","291,907",AKR,0
76,31-Mar-2024,"670,841","1,468","1,468","586,219","363,996","1,177,226","1,471,533",0,"58,890",...,"1,510,652","22,075",0,"207,202","97,226","25,649","1,177,226","329,873",AKR,0


In [ ]:
def compare_columns_in_dict(dfs_dict):
  results = {}
  first_df_columns = sorted(list(dfs_dict.values())[0].columns)  # Get columns of the first DataFrame

  for key, df in dfs_dict.items():
    current_df_columns = sorted(df.columns)
    if current_df_columns == first_df_columns:
      results[key] = "Yes"
    else:
      results[key] = "No"

  return results

# Example usage:
column_comparison_results = compare_columns_in_dict(bs_dfs)

for file, result in column_comparison_results.items():
  print(f"File: {file}, Columns Same: {result}")

File: AKR.balancesheet (STARK).xls.csv, Columns Same: Yes
File: ACE.balancesheet (EA).csv, Columns Same: Yes
File: BBGI.balancesheet (EA).csv, Columns Same: Yes
File: BCPG.balancesheet (EA).csv, Columns Same: Yes
File: CKP.balancesheet (EA).csv, Columns Same: Yes
File: CPT.balancesheet (STARK).csv, Columns Same: Yes
File: ETC.balancesheet (EA).csv, Columns Same: Yes
File: EA.balancesheet.csv, Columns Same: Yes
File: SCI.balancesheet (STARK).csv, Columns Same: Yes
File: LNE.balancesheet (STARK).csv, Columns Same: Yes
File: SSP.balancesheet (EA).csv, Columns Same: Yes
File: SPCG.balancesheet (EA).csv, Columns Same: Yes
File: SUPER.balancesheet (EA).csv, Columns Same: Yes
File: STARK.balancesheet.csv, Columns Same: Yes
File: TMIm.balancesheet (STARK).csv, Columns Same: Yes
File: TGE.balancesheet (EA).csv, Columns Same: Yes
File: TRTm.balancesheet (STARK).csv, Columns Same: Yes
File: TPIPP.balancesheet (EA).csv, Columns Same: Yes


In [ ]:
bs_dfs_merged = pd.concat(bs_dfs.values(), ignore_index=True)
print(len(bs_dfs_merged.columns.to_list()))
bs_dfs_merged

45


,Period End Date,Total Current Liabilities,Minority Interest,Minority Interest - Non Redeemable,Total Current Assets less Inventory,"Accounts Receivable - Trade, Gross",Common Stock,Total Common Shares Outstanding,Receivables - Other,Defered Income Tax - Long Term Asset,...,"Tangible Book Value, Common Equity",Long Term Investments,Treas Shares - Common Stock Prmry Issue,Cash and Short Term Investments,"Other Current liabilities, Total",Additional Paid-In Capital,"Common Stock, Total",Retained Earnings (Accumulated Deficit),Company name,Fraud
0,31-Dec-2004,"821,500","(179,534)","(179,534)","447,718","425,090","508,174","1,002,140","4,409",0,...,"213,202",0,0,"33,950","190,948","107,500","508,174","(167,701)",AKR,0
1,30-Jun-2005,"626,215",0,0,"492,350","418,756","608,174","1,002,140","14,823",0,...,"513,794",0,0,"38,295","60,269","207,500","608,174","(96,783)",AKR,0
2,31-Dec-2005,"703,892",0,0,"410,495","326,237","608,174","1,002,140","21,000",0,...,"620,668",0,0,"45,014","77,238","4,924","608,174","7,570",AKR,0
3,31-Mar-2006,"785,650",0,0,"417,965","291,767","608,174","1,002,140","11,848",0,...,"624,519",0,0,"87,349","102,648","4,924","608,174","11,421",AKR,0
4,30-Jun-2006,"764,503",0,0,"476,753","389,935","608,174","1,002,140","12,731",0,...,"669,173",0,0,"46,049","98,828","4,924","608,174","56,075",AKR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,30-Jun-2023,"4,864,801",4,4,"9,247,289","2,677,737","8,400,000","8,400,000","277,318","79,160",...,"32,537,824","737,908",0,"6,259,700","1,123,119","14,580,886","8,400,000","9,556,938",TPIPP,0
847,30-Sep-2023,"9,364,555",4,4,"7,417,569","1,650,596","8,400,000","8,400,000","364,485","76,710",...,"32,368,559","739,262",0,"5,318,738","1,874,095","14,580,886","8,400,000","9,387,673",TPIPP,0
848,31-Dec-2023,"6,791,053",4,4,"6,658,315","1,726,988","8,400,000","8,400,000","332,201","76,170",...,"33,201,585","740,180",0,"4,419,304","2,279,995","14,580,886","8,400,000","10,220,699",TPIPP,0
849,31-Mar-2024,"7,635,186",4,4,"6,289,334","1,831,224","8,400,000","8,400,000","344,941","70,744",...,"32,893,000","741,834",0,"3,969,052","2,923,848","14,580,886","8,400,000","9,912,114",TPIPP,0


In [ ]:
bs_df = bs_dfs_merged
bs_df

,Period End Date,Total Current Liabilities,Minority Interest,Minority Interest - Non Redeemable,Total Current Assets less Inventory,"Accounts Receivable - Trade, Gross",Common Stock,Total Common Shares Outstanding,Receivables - Other,Defered Income Tax - Long Term Asset,...,"Tangible Book Value, Common Equity",Long Term Investments,Treas Shares - Common Stock Prmry Issue,Cash and Short Term Investments,"Other Current liabilities, Total",Additional Paid-In Capital,"Common Stock, Total",Retained Earnings (Accumulated Deficit),Company name,Fraud
0,31-Dec-2004,"821,500","(179,534)","(179,534)","447,718","425,090","508,174","1,002,140","4,409",0,...,"213,202",0,0,"33,950","190,948","107,500","508,174","(167,701)",AKR,0
1,30-Jun-2005,"626,215",0,0,"492,350","418,756","608,174","1,002,140","14,823",0,...,"513,794",0,0,"38,295","60,269","207,500","608,174","(96,783)",AKR,0
2,31-Dec-2005,"703,892",0,0,"410,495","326,237","608,174","1,002,140","21,000",0,...,"620,668",0,0,"45,014","77,238","4,924","608,174","7,570",AKR,0
3,31-Mar-2006,"785,650",0,0,"417,965","291,767","608,174","1,002,140","11,848",0,...,"624,519",0,0,"87,349","102,648","4,924","608,174","11,421",AKR,0
4,30-Jun-2006,"764,503",0,0,"476,753","389,935","608,174","1,002,140","12,731",0,...,"669,173",0,0,"46,049","98,828","4,924","608,174","56,075",AKR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,30-Jun-2023,"4,864,801",4,4,"9,247,289","2,677,737","8,400,000","8,400,000","277,318","79,160",...,"32,537,824","737,908",0,"6,259,700","1,123,119","14,580,886","8,400,000","9,556,938",TPIPP,0
847,30-Sep-2023,"9,364,555",4,4,"7,417,569","1,650,596","8,400,000","8,400,000","364,485","76,710",...,"32,368,559","739,262",0,"5,318,738","1,874,095","14,580,886","8,400,000","9,387,673",TPIPP,0
848,31-Dec-2023,"6,791,053",4,4,"6,658,315","1,726,988","8,400,000","8,400,000","332,201","76,170",...,"33,201,585","740,180",0,"4,419,304","2,279,995","14,580,886","8,400,000","10,220,699",TPIPP,0
849,31-Mar-2024,"7,635,186",4,4,"6,289,334","1,831,224","8,400,000","8,400,000","344,941","70,744",...,"32,893,000","741,834",0,"3,969,052","2,923,848","14,580,886","8,400,000","9,912,114",TPIPP,0


In [ ]:
bs_df.dtypes

,0
0,
Period End Date,object
Total Current Liabilities,object
Minority Interest,object
Minority Interest - Non Redeemable,object
Total Current Assets less Inventory,object
"Accounts Receivable - Trade, Gross",object
Common Stock,object
Total Common Shares Outstanding,object
Receivables - Other,object


In [ ]:
column_names = bs_df.columns
blacklist = ["Period End Date","Company name","Fraud"]
for col in column_names:
    if col not in blacklist:
      bs_df[col] = bs_df[col].apply(acc_tran)

bs_df

,Period End Date,Total Current Liabilities,Minority Interest,Minority Interest - Non Redeemable,Total Current Assets less Inventory,"Accounts Receivable - Trade, Gross",Common Stock,Total Common Shares Outstanding,Receivables - Other,Defered Income Tax - Long Term Asset,...,"Tangible Book Value, Common Equity",Long Term Investments,Treas Shares - Common Stock Prmry Issue,Cash and Short Term Investments,"Other Current liabilities, Total",Additional Paid-In Capital,"Common Stock, Total",Retained Earnings (Accumulated Deficit),Company name,Fraud
0,31-Dec-2004,821500.0,-179534.0,-179534.0,447718.0,425090.0,508174.0,1002140.0,4409.0,0.0,...,213202.0,0.0,0.0,33950.0,190948.0,107500.0,508174.0,-167701.0,AKR,0
1,30-Jun-2005,626215.0,0.0,0.0,492350.0,418756.0,608174.0,1002140.0,14823.0,0.0,...,513794.0,0.0,0.0,38295.0,60269.0,207500.0,608174.0,-96783.0,AKR,0
2,31-Dec-2005,703892.0,0.0,0.0,410495.0,326237.0,608174.0,1002140.0,21000.0,0.0,...,620668.0,0.0,0.0,45014.0,77238.0,4924.0,608174.0,7570.0,AKR,0
3,31-Mar-2006,785650.0,0.0,0.0,417965.0,291767.0,608174.0,1002140.0,11848.0,0.0,...,624519.0,0.0,0.0,87349.0,102648.0,4924.0,608174.0,11421.0,AKR,0
4,30-Jun-2006,764503.0,0.0,0.0,476753.0,389935.0,608174.0,1002140.0,12731.0,0.0,...,669173.0,0.0,0.0,46049.0,98828.0,4924.0,608174.0,56075.0,AKR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,30-Jun-2023,4864801.0,4.0,4.0,9247289.0,2677737.0,8400000.0,8400000.0,277318.0,79160.0,...,32537824.0,737908.0,0.0,6259700.0,1123119.0,14580886.0,8400000.0,9556938.0,TPIPP,0
847,30-Sep-2023,9364555.0,4.0,4.0,7417569.0,1650596.0,8400000.0,8400000.0,364485.0,76710.0,...,32368559.0,739262.0,0.0,5318738.0,1874095.0,14580886.0,8400000.0,9387673.0,TPIPP,0
848,31-Dec-2023,6791053.0,4.0,4.0,6658315.0,1726988.0,8400000.0,8400000.0,332201.0,76170.0,...,33201585.0,740180.0,0.0,4419304.0,2279995.0,14580886.0,8400000.0,10220699.0,TPIPP,0
849,31-Mar-2024,7635186.0,4.0,4.0,6289334.0,1831224.0,8400000.0,8400000.0,344941.0,70744.0,...,32893000.0,741834.0,0.0,3969052.0,2923848.0,14580886.0,8400000.0,9912114.0,TPIPP,0


In [ ]:
bs_df.dtypes

,0
0,
Period End Date,object
Total Current Liabilities,float64
Minority Interest,float64
Minority Interest - Non Redeemable,float64
Total Current Assets less Inventory,float64
"Accounts Receivable - Trade, Gross",float64
Common Stock,float64
Total Common Shares Outstanding,float64
Receivables - Other,float64


In [ ]:
bs_df.to_csv('bs_df.csv', index=False)

## Incomesheet DataFrame

In [ ]:
is_files = [f for f in all_files if f.endswith('.csv') and 'incomesheet' in f]

is_dfs = {}
for file in is_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if 'Period End Date' in line:
                skiprows = i
                break
        else:
            skiprows = 0  # If 'Period End Date' not found, don't skip any rows
    is_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None)

    # Transpose
    is_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None).transpose()
    # Shift first row to header
    is_dfs[file].columns = is_dfs[file].iloc[0]
    is_dfs[file] = is_dfs[file][1:]

In [ ]:
print(len(is_dfs.keys()))
is_dfs.keys()

18


dict_keys(['AKR.incomesheet (STARK).xls.csv', 'ACE.incomesheet (EA).csv', 'BBGI.incomesheet (EA).csv', 'BCPG.incomesheet (EA).csv', 'CKP.incomesheet (EA).csv', 'CPT.incomesheet (STARK).csv', 'ETC.incomesheet (EA).csv', 'EA.incomesheet.csv', 'SCI.incomesheet (STARK).csv', 'LNE.incomesheet (STARK).csv', 'SSP.incomesheet (EA).csv', 'SPCG.incomesheet (EA).csv', 'SUPER.incomesheet (EA).csv', 'STARK.incomesheet.csv', 'TMIm.incomesheet (STARK).csv', 'TGE.incomesheet (EA).csv', 'TPIPP.incomesheet (EA).csv', 'TRTm.incomesheet (STARK).csv'])

In [ ]:
# Get the columns for the first DataFrame as a starting point
all_columns = set(is_dfs[list(is_dfs.keys())[0]].columns)

# Find common columns (intersection)
common_columns = all_columns.intersection(*[set(df.columns) for df in is_dfs.values()])

# Find different columns (unique to each DataFrame)
different_columns = {}
for key, df in is_dfs.items():
    unique_cols = set(df.columns) - common_columns
    if unique_cols:  # Add only if there are unique columns
        different_columns[key] = list(unique_cols)
print("-" * 20)
# Print the results
print("Common Columns:", list(common_columns))
print("\nDifferent Columns (Unique to each DataFrame):")
for key, cols in different_columns.items():
    print(f"{key}: {cols}")
print("-" * 20)
# Count common and different columns
num_common_cols = len(common_columns)
num_different_cols = sum(len(cols) for cols in different_columns.values())
print()
# Print the results
print(f"Number of Common Columns: {num_common_cols}")
print(f"Number of Different Columns: {num_different_cols}")
print("-" * 20)

--------------------
Common Columns: ['Cost of Revenue', nan, 'Normalized Income Before Taxes', 'Diluted Weighted Average Shares', 'Inc Tax Ex Impact of Sp Items', 'Depreciation, Supplemental', 'Normalized EBIT', 'Net Income', 'Net Income After Taxes', 'Income Available to Com Excl ExtraOrd', 'Period End Date', 'Minority Interest, Supplemental', 'Cash Dividend', 'Basic Normalized EPS', 'Other Operating Expenses, Total', 'Basic Weighted Average Shares', 'Cost of Revenue, Total', 'Dividends and Capital Changes', 'Normalized EBITDA', 'Supplemental (THB Thousands)', 'Interest Inc.(Exp.),Net-Non-Op., Total', 'Diluted EPS Excluding ExtraOrd Items', 'Other, Net', 'Normalized Income After Taxes', 'Minority Interest', 'Basic EPS Including Extraordinary Items', 'Operating Income', 'Income Available to Com Incl ExtraOrd', 'Net Income Before Taxes', 'Basic EPS Excluding Extraordinary Items', 'DPS - Common Stock Primary Issue', 'Provision for Income Taxes', 'Diluted Normalized EPS', 'Normalized Inc

In [ ]:
# Drop different columns in each DataFrame and update is_dfs
for key in is_dfs:
    is_dfs[key] = is_dfs[key][list(common_columns)]

In [ ]:
for key in is_dfs:
    is_dfs[key] = is_dfs[key].replace('--', 0)
    is_dfs[key] = is_dfs[key].replace([' ', '  ', np.nan, None], np.nan)

<ipython-input-28-4d13572594e0>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_dfs[key] = is_dfs[key].replace('--', 0)


In [ ]:
for key, df in is_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.incomesheet (STARK).xls.csv
Column 'nan' has 78 NaN values.
Column 'nan' has 78 NaN values.
Column 'nan' has 78 NaN values.
Column 'nan' has 58 NaN values.
Column 'nan' has 75 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 77 NaN values.
Column 'nan' has 76 NaN values.
Column 'Dividends and Capital Changes' has 78 NaN values.
Column 'Supplemental (THB Thousands)' has 78 NaN values.
--------------------
File: ACE.incomesheet (EA).csv
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'nan' has 26 NaN values.
Column 'nan' has 20 NaN values.
Column 'nan' has 25 NaN values.
Column 'Dividends and Capital Changes' has 26 NaN values.
Column 'Supplemental (THB Thousands)' has 26 NaN values.
--------------------
File: BBGI.incomesheet (EA).csv
Column 'nan' has 14 NaN values.
Column 'nan' has 14 NaN values.
Column 'nan' has 14 NaN values.
Column 'nan' has 10 NaN values.
Column 'Dividends a

In [ ]:
is_dfs["TMIm.incomesheet (STARK).csv"] = is_dfs["TMIm.incomesheet (STARK).csv"].iloc[1:]
is_dfs["STARK.incomesheet.csv"] = is_dfs["STARK.incomesheet.csv"].iloc[1:]

In [ ]:
for key, df in is_dfs.items():
    columns_to_drop = ['Supplemental (THB Thousands)',
                       'Dividends and Capital Changes',
                       'Other, Net',
                       'Cash Dividend',
                       'nan','NaN']

    # Drop columns if they exist in the DataFrame
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=1)

    # Update the DataFrame in is_dfs
    is_dfs[key] = df

In [ ]:
for key, df in is_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

File: AKR.incomesheet (STARK).xls.csv
--------------------
File: ACE.incomesheet (EA).csv
--------------------
File: BBGI.incomesheet (EA).csv
--------------------
File: BCPG.incomesheet (EA).csv
--------------------
File: CKP.incomesheet (EA).csv
--------------------
File: CPT.incomesheet (STARK).csv
--------------------
File: ETC.incomesheet (EA).csv
--------------------
File: EA.incomesheet.csv
--------------------
File: SCI.incomesheet (STARK).csv
--------------------
File: LNE.incomesheet (STARK).csv
--------------------
File: SSP.incomesheet (EA).csv
--------------------
File: SPCG.incomesheet (EA).csv
--------------------
File: SUPER.incomesheet (EA).csv
--------------------
File: STARK.incomesheet.csv
--------------------
File: TMIm.incomesheet (STARK).csv
--------------------
File: TGE.incomesheet (EA).csv
--------------------
File: TPIPP.incomesheet (EA).csv
--------------------
File: TRTm.incomesheet (STARK).csv
--------------------


In [ ]:
is_dfs["TPIPP.incomesheet (EA).csv"]

,Cost of Revenue,NaN,Normalized Income Before Taxes,Diluted Weighted Average Shares,Inc Tax Ex Impact of Sp Items,"Depreciation, Supplemental",Normalized EBIT,Net Income,Net Income After Taxes,Income Available to Com Excl ExtraOrd,...,Diluted Normalized EPS,Normalized Inc. Avail to Com.,Total Revenue,Diluted Net Income,Total Operating Expense,"Selling/General/Admin. Expenses, Total",Selling/General/Administrative Expense,Revenue,Diluted EPS Including ExtraOrd Items,Net Income Before Extra. Items
1,"409,877",0,"183,324","4,369,722","(3,514)",0,"169,224","186,838","186,838","186,838",...,0.04,"186,838","591,691","186,838","422,467","21,085","21,085","591,691",0.04,"186,838"
2,"363,073",0,"190,639","4,442,039","1,182",0,"210,201","189,457","189,457","189,457",...,0.04,"189,457","565,434","189,457","355,233","26,800","26,800","565,434",0.04,"189,457"
3,"503,608",0,"634,160","5,900,000","(2,097)",0,"590,471","636,257","636,257","636,257",...,0.11,"636,257","1,135,308","636,257","544,837","48,715","48,715","1,135,308",0.11,"636,257"
4,"548,641",0,"498,615","5,900,000","(1,073)",0,"532,963","499,688","499,688","499,688",...,0.08,"499,688","1,120,224","499,688","587,261","47,872","47,872","1,120,224",0.08,"499,688"
5,"546,620",0,"282,072","5,900,000","(2,159)","89,248","305,120","284,231","284,231","284,231",...,0.05,"284,231","897,805","284,231","592,685","53,794","53,794","897,805",0.05,"284,231"
6,"666,140",0,"402,496","5,900,000","(1,576)","93,526","517,979","404,072","404,072","404,072",...,0.07,"404,072","1,214,823","404,072","696,844","40,199","40,199","1,214,823",0.07,"404,072"
7,"475,499",0,"701,109","5,959,444","2,616","84,470","645,711","698,493","698,493","698,493",...,0.12,"698,493","1,166,550","698,493","520,839","47,889","47,889","1,166,550",0.12,"698,493"
8,"533,848",0,"716,284","8,400,000","9,920","91,356","667,935","706,364","706,364","706,364",...,0.08,"706,364","1,241,125","706,364","573,190","49,716","49,716","1,241,125",0.08,"706,364"
9,"535,033",THB 0.04 Extra,"619,389","8,400,000","26,035","95,840","554,615","593,354","593,354","593,354",...,0.07,"593,354","1,143,190","593,354","588,575","77,229","77,229","1,143,190",0.07,"593,354"
10,"683,155",THB 0.02 Extra,"630,329","8,406,698","37,009","92,773","565,848","593,320","593,320","593,320",...,0.07,"593,320","1,313,949","593,320","748,101","77,757","77,757","1,313,949",0.07,"593,320"


In [ ]:
# specific_files = ['AKR.incomesheet (STARK).xls.csv', 'BCPG.incomesheet (EA).csv', 'ETC.incomesheet (EA).csv', 'EA.incomesheet.csv', 'SSP.incomesheet (EA).csv', 'TPIPP.incomesheet (EA).csv']

# for key in specific_files:
#     if key in is_dfs:  # Check if the key exists in is_dfs
#         is_dfs[key] = is_dfs[key].iloc[:, 1:]

In [ ]:
for key, df in is_dfs.items():
    print(f"DataFrame: {key}, Shape: {df.shape}")

DataFrame: AKR.incomesheet (STARK).xls.csv, Shape: (78, 37)
DataFrame: ACE.incomesheet (EA).csv, Shape: (26, 36)
DataFrame: BBGI.incomesheet (EA).csv, Shape: (14, 36)
DataFrame: BCPG.incomesheet (EA).csv, Shape: (38, 37)
DataFrame: CKP.incomesheet (EA).csv, Shape: (50, 36)
DataFrame: CPT.incomesheet (STARK).csv, Shape: (34, 36)
DataFrame: ETC.incomesheet (EA).csv, Shape: (20, 37)
DataFrame: EA.incomesheet.csv, Shape: (59, 37)
DataFrame: SCI.incomesheet (STARK).csv, Shape: (42, 36)
DataFrame: LNE.incomesheet (STARK).csv, Shape: (86, 36)
DataFrame: SSP.incomesheet (EA).csv, Shape: (34, 37)
DataFrame: SPCG.incomesheet (EA).csv, Shape: (82, 36)
DataFrame: SUPER.incomesheet (EA).csv, Shape: (82, 36)
DataFrame: STARK.incomesheet.csv, Shape: (76, 36)
DataFrame: TMIm.incomesheet (STARK).csv, Shape: (58, 36)
DataFrame: TGE.incomesheet (EA).csv, Shape: (16, 36)
DataFrame: TPIPP.incomesheet (EA).csv, Shape: (36, 37)
DataFrame: TRTm.incomesheet (STARK).csv, Shape: (77, 36)


In [ ]:
for key in is_dfs:
    # Get a list of columns, with 'Period End Date' moved to the front
    cols = ['Period End Date'] + [col for col in is_dfs[key].columns if col != 'Period End Date']

    # Reorder the DataFrame columns
    is_dfs[key] = is_dfs[key][cols]

In [ ]:
# Get the column names from the first DataFrame as the reference order
reference_columns = is_dfs[list(is_dfs.keys())[0]].columns

# Reorder columns in all DataFrames to match the reference order
for key in is_dfs:
    is_dfs[key] = is_dfs[key][reference_columns]

KeyError: '[nan] not in index'

In [ ]:
def compare_columns_in_dict(dfs_dict):
  results = {}
  first_df_columns = sorted(list(dfs_dict.values())[0].columns)  # Get columns of the first DataFrame

  for key, df in dfs_dict.items():
    current_df_columns = sorted(df.columns)
    if current_df_columns == first_df_columns:
      results[key] = "Yes"
    else:
      results[key] = "No"

  return results

# Example usage:
column_comparison_results = compare_columns_in_dict(is_dfs)

for file, result in column_comparison_results.items():
  print(f"File: {file}, Columns Same: {result}")

TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
for key in is_dfs:
    # Extract company name using regular expression
    company_name = re.search(r'(.*)\.incomesheet', key).group(1)

    # Add 'Company name' column
    is_dfs[key]['Company name'] = company_name

In [ ]:
for key in is_dfs:
    # Add 'Fraud' column and initialize to 0
    is_dfs[key]['Fraud'] = 0

    # Set 'Fraud' to 1 for specific companies
    company_name = is_dfs[key]['Company name'].iloc[0]  # Get company name
    if company_name in ['EA', 'STARK']:
        is_dfs[key]['Fraud'] = 1

In [ ]:
is_dfs_merged = pd.concat(is_dfs.values(), ignore_index=True)
print(len(is_dfs_merged.columns.to_list()))
is_dfs_merged

In [ ]:
is_df = is_dfs_merged
is_df

In [ ]:
column_names = is_df.columns
blacklist = ["Period End Date","Company name","Fraud"]
for col in column_names:
    if col not in blacklist:
      is_df[col] = is_df[col].apply(acc_tran)

print(is_df)

In [ ]:
is_df.dtypes

In [ ]:
is_df.to_csv('is_df.csv', index=False)

## Ratios-key Metrics DataFrame

In [ ]:
ratio_files = [f for f in all_files if f.endswith('.csv') and 'ratios-key metrics' in f]

ratio_dfs = {}

ratio_dfs = {}
for file in ratio_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if 'Industry Median' in line:
                skiprows = i
                break
        else:
            skiprows = 0  # If 'Period End Date' not found, don't skip any rows
    ratio_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None)

    # Transpose
    ratio_dfs[file] = pd.read_csv(file_path, skiprows=skiprows, header=None).transpose()
    # Shift first row to header
    ratio_dfs[file].columns = ratio_dfs[file].iloc[0]
    ratio_dfs[file] = ratio_dfs[file][1:]

In [ ]:
print(len(ratio_dfs.keys()))
ratio_dfs.keys()

In [ ]:
# Get the columns for the first DataFrame as a starting point
all_columns = set(ratio_dfs[list(ratio_dfs.keys())[0]].columns)

# Find common columns (intersection)
common_columns = all_columns.intersection(*[set(df.columns) for df in ratio_dfs.values()])

# Find different columns (unique to each DataFrame)
different_columns = {}
for key, df in ratio_dfs.items():
    unique_cols = set(df.columns) - common_columns
    if unique_cols:  # Add only if there are unique columns
        different_columns[key] = list(unique_cols)

# Print the results
print("Common Columns:", list(common_columns))
print("\nDifferent Columns (Unique to each DataFrame):")
for key, cols in different_columns.items():
    print(f"{key}: {cols}")
print("-" * 20)
# Count common and different columns
num_common_cols = len(common_columns)
num_different_cols = sum(len(cols) for cols in different_columns.values())
print("-" * 20)
# Print the results
print(f"Number of Common Columns: {num_common_cols}")
print(f"Number of Different Columns: {num_different_cols}")

In [ ]:
# Drop different columns in each DataFrame and update ratio_dfs
for key in ratio_dfs:
    ratio_dfs[key] = ratio_dfs[key][list(common_columns)]

In [ ]:
for key in ratio_dfs:
    ratio_dfs[key] = ratio_dfs[key].replace(['--','-'], 0)
    ratio_dfs[key] = ratio_dfs[key].replace([' ', '  ', np.nan, None], np.nan)

In [ ]:
for key in ratio_dfs:
    ratio_dfs[key] = ratio_dfs[key].iloc[1:]

In [ ]:
for key, df in ratio_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

In [ ]:
for key, df in ratio_dfs.items():
    columns_to_drop = ['Profitability',
                      'Liquidity',
                      'Leverage',
                       'DuPont/Earning Power']

    # Drop columns if they exist in the DataFrame
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    df = df.dropna(axis=1, how='all')

    # Update the DataFrame in cf_dfs
    ratio_dfs[key] = df

In [ ]:
for key, df in ratio_dfs.items():
    print(f"File: {key}")
    nan_counts = df.isna().sum()
    columns_with_na = nan_counts[nan_counts > 0]
    for column, count in columns_with_na.items():
        print(f"Column '{column}' has {count} NaN values.")
    print("-" * 20)  # Separator between files

In [ ]:
for key, df in ratio_dfs.items():
    # Get the current column names
    columns = df.columns.tolist()

    # Force the first column to be 'Period End Date'
    columns[0] = 'Period End Date'
    df.columns = columns  # Update the column names

In [ ]:
# Get the column names from the first DataFrame as the reference order
reference_columns = ratio_dfs[list(ratio_dfs.keys())[0]].columns

# Reorder columns in all DataFrames to match the reference order
for key in ratio_dfs:
    ratio_dfs[key] =ratio_dfs[key][reference_columns]

In [ ]:
def compare_columns_in_dict(dfs_dict):
  results = {}
  first_df_columns = sorted(list(dfs_dict.values())[0].columns)  # Get columns of the first DataFrame

  for key, df in dfs_dict.items():
    current_df_columns = sorted(df.columns)
    if current_df_columns == first_df_columns:
      results[key] = "Yes"
    else:
      results[key] = "No"

  return results

# Example usage:
column_comparison_results = compare_columns_in_dict(ratio_dfs)

for file, result in column_comparison_results.items():
  print(f"File: {file}, Columns Same: {result}")

In [ ]:
for key in ratio_dfs:
    # Extract company name using regular expression
    company_name = re.search(r'(.*)\.ratios-key metrics', key).group(1)

    # Add 'Company name' column
    ratio_dfs[key]['Company name'] = company_name

In [ ]:
for key in ratio_dfs:
    # Add 'Fraud' column and initialize to 0
    ratio_dfs[key]['Fraud'] = 0

    # Set 'Fraud' to 1 for specific companies
    company_name = ratio_dfs[key]['Company name'].iloc[0]  # Get company name
    if company_name in ['EA', 'STARK']:
        ratio_dfs[key]['Fraud'] = 1

In [ ]:
for key, df in ratio_dfs.items():
    print(f"File: {key}")
    print(f"Shape: {df.shape}")  # Print the shape (rows, columns)
    print("-" * 20)  # Separator between files

In [ ]:
ratio_dfs_merged = pd.concat(ratio_dfs.values(), ignore_index=True)
print(len(ratio_dfs_merged.columns.to_list()))
ratio_dfs_merged

In [ ]:
ratio_dfs_merged.info()

In [ ]:
ratio_df = ratio_dfs_merged
ratio_df

In [ ]:
def percentage_to_float(percentage_str):
    """
    Converts a percentage string (e.g., "17.5%", "(255.0%)") to a float.

    Handles negative values indicated by parentheses.
    """
    # Check if the input is already a number (int or float)
    if isinstance(percentage_str, (int, float)):
        return percentage_str

    try:
        # Remove percentage sign and commas
        percentage_str = percentage_str.replace("%", "").replace(",", "")
        # Handle negative values in parentheses
        if percentage_str.startswith("(") and percentage_str.endswith(")"):
            percentage_str = percentage_str[1:-1]  # Remove parentheses
            return float(percentage_str) * -1  # Convert to float and multiply by -1
        else:
            return float(percentage_str)  # Convert to float
    except (ValueError, TypeError, AttributeError):
        # Handle cases where the input is not a valid percentage string
        return None  # Or raise an exception if you prefer

In [ ]:
column_names = ratio_df.columns
blacklist = ["Period End Date","Company name","Fraud"]
for col in column_names:
    if col not in blacklist:
      ratio_df[col] = ratio_df[col].apply(percentage_to_float)

print(ratio_df)

In [ ]:
ratio_df.dtypes

In [ ]:
ratio_df.to_csv('ratio_df.csv', index=False)

# Model?

In [ ]:
ph_df = pd.read_csv("/content/drive/MyDrive/EC439/ph_df.csv")
bs_df = pd.read_csv("/content/drive/MyDrive/EC439/bs_df.csv")
cf_df = pd.read_csv("/content/drive/MyDrive/EC439/cf_df.csv")
is_df = pd.read_csv("/content/drive/MyDrive/EC439/is_df.csv")
ratio_df = pd.read_csv("/content/drive/MyDrive/EC439/ratio_df.csv")

## Normalization except ph_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalize_by_group(df, group_col, cols_to_normalize):
    """
    Normalizes specified columns within groups of a DataFrame.

    Args:
        df: DataFrame to normalize.
        group_col: Column name to group by.
        cols_to_normalize: List of column names to normalize.

    Returns:
        DataFrame with normalized columns.
    """

    # Create a copy of the DataFrame to avoid modifying the original
    df_normalized = df.copy()

    # Group the DataFrame by the specified column
    for group, group_data in df_normalized.groupby(group_col):
        # Create a MinMaxScaler object
        scaler = MinMaxScaler()

        # Fit the scaler to the group data for the specified columns
        scaler.fit(group_data[cols_to_normalize])

        # Transform the group data and update the original DataFrame
        df_normalized.loc[group_data.index, cols_to_normalize] = scaler.transform(group_data[cols_to_normalize])

    return df_normalized

In [ ]:
# Get columns to normalize (excluding 'Company name', 'Period End Date', 'Fraud')
cols_to_normalize = [col for col in bs_df.columns if col not in ['Company name', 'Period End Date', 'Fraud']]

# Normalize bs_df
bs_df_normalized = normalize_by_group(bs_df, 'Company name', cols_to_normalize)

# Display the normalized DataFrame
bs_df_normalized.head()

,Period End Date,Total Current Liabilities,Minority Interest,Minority Interest - Non Redeemable,Total Current Assets less Inventory,"Accounts Receivable - Trade, Gross",Common Stock,Total Common Shares Outstanding,Receivables - Other,Defered Income Tax - Long Term Asset,...,"Tangible Book Value, Common Equity",Long Term Investments,Treas Shares - Common Stock Prmry Issue,Cash and Short Term Investments,"Other Current liabilities, Total",Additional Paid-In Capital,"Common Stock, Total",Retained Earnings (Accumulated Deficit),Company name,Fraud
0,31-Dec-2004,0.332084,0.00000,0.00000,0.388249,0.648119,0.0000,0.000017,0.047198,0.0,...,0.000000,0.0,0.0,0.071515,0.665984,0.362463,0.0000,0.567570,AKR,0
1,30-Jun-2005,0.217698,0.99189,0.99189,0.473400,0.631017,0.1494,0.000017,0.158681,0.0,...,0.231679,0.0,0.0,0.088396,0.113990,0.699638,0.1494,0.629203,AKR,0
2,31-Dec-2005,0.263196,0.99189,0.99189,0.317233,0.381211,0.1494,0.000017,0.224806,0.0,...,0.314051,0.0,0.0,0.114501,0.185668,0.016602,0.1494,0.719894,AKR,0
3,31-Mar-2006,0.311085,0.99189,0.99189,0.331485,0.288141,0.1494,0.000017,0.126833,0.0,...,0.317020,0.0,0.0,0.278982,0.293001,0.016602,0.1494,0.723241,AKR,0
4,30-Jun-2006,0.298699,0.99189,0.99189,0.443643,0.553199,0.1494,0.000017,0.136286,0.0,...,0.351436,0.0,0.0,0.118522,0.276865,0.016602,0.1494,0.762049,AKR,0


In [ ]:
# Get columns to normalize (excluding 'Company name', 'Period End Date', 'Fraud')
cols_to_normalize = [col for col in cf_df.columns if col not in ['Company name', 'Period End Date', 'Fraud']]

# Normalize cf_df
cf_df_normalized = normalize_by_group(cf_df, 'Company name', cols_to_normalize)

# Display the normalized DataFrame
cf_df_normalized.head()

,Period End Date,Non-Cash Items,"Issuance (Retirement) of Debt, Net",Capital Expenditures,Other Liabilities,Other Financing Cash Flow,Other Investing Cash Flow,Net Change in Cash,Depreciation/Depletion,Cash Interest Paid,...,Cash from Investing Activities,Net Income/Starting Line,"Long Term Debt, Net","Other Investing Cash Flow Items, Total",Cash Taxes Paid,Net Changes in Working Capital,Free Cash Flow,Other Non-Cash Items,Company name,Fraud
0,30-Jun-2004,0.793638,0.340841,0.917347,1.000000,1.0,0.676724,0.621870,0.437577,0.005096,...,0.917157,0.217011,0.422855,0.622023,0.112915,0.493514,0.799210,0.048771,AKR,0
1,31-Dec-2004,0.000000,0.385589,0.830766,0.975373,1.0,0.676724,0.547415,0.927476,0.254021,...,0.830697,1.000000,0.442545,0.624011,0.137158,0.428646,0.723606,0.023713,AKR,0
2,31-Mar-2005,0.797495,0.263162,0.917212,0.279882,1.0,0.000000,0.474524,0.000000,0.022968,...,0.910188,0.186277,0.269855,0.573118,0.097901,0.433717,0.667594,0.080311,AKR,0
3,30-Jun-2005,0.797220,0.350966,0.904728,0.396051,1.0,0.000000,0.518134,0.088752,0.056369,...,0.897681,0.221318,0.255835,0.573118,0.106829,0.180858,0.573833,0.077710,AKR,0
4,30-Sep-2005,0.802723,0.315637,0.888775,0.404352,1.0,0.000000,0.541272,0.178662,0.131473,...,0.881699,0.253348,0.244482,0.573118,0.137068,0.162100,0.624706,0.130190,AKR,0


In [ ]:
# Get columns to normalize (excluding 'Company name', 'Period End Date', 'Fraud')
cols_to_normalize = [col for col in is_df.columns if col not in ['Company name', 'Period End Date', 'Fraud']]

# Normalize is_df
is_df_normalized = normalize_by_group(is_df, 'Company name', cols_to_normalize)

# Display the normalized DataFrame
is_df_normalized.head()

,Period End Date,Revenue,Cost of Revenue,Operating Income,Normalized Income After Taxes,Net Income After Taxes,"Other Operating Expenses, Total","Interest Inc.(Exp.),Net-Non-Op., Total",Net Income Before Taxes,Net Income Before Extra. Items,...,Basic Weighted Average Shares,Diluted EPS Excluding ExtraOrd Items,Normalized EBITDA,Selling/General/Administrative Expense,Income Available to Com Excl ExtraOrd,Inc Tax Ex Impact of Sp Items,"Cost of Revenue, Total",Diluted Normalized EPS,Company name,Fraud
0,31-Mar-2005,0.001338,0.043705,0.392702,0.174943,0.362236,0.642311,0.954776,0.408557,0.362236,...,0.493822,0.323077,0.064341,0.000000,0.362236,0.625886,0.043705,0.225,AKR,0
1,30-Jun-2005,0.289318,0.252390,0.465973,0.299749,0.445969,0.731811,0.936631,0.483752,0.445969,...,0.493822,0.430769,0.208742,0.208325,0.445969,0.624327,0.252390,0.400,AKR,0
2,30-Sep-2005,0.312797,0.285337,0.461544,0.288596,0.438487,0.524855,0.896523,0.477234,0.438487,...,0.493822,0.415385,0.200925,0.285160,0.438487,0.625107,0.285337,0.375,AKR,0
3,31-Dec-2005,0.347294,0.409249,0.445574,0.261889,0.420568,0.079010,0.904510,0.461038,0.420568,...,0.493822,0.400000,0.172967,0.200430,0.420568,0.625107,0.409249,0.350,AKR,0
4,31-Mar-2006,0.180946,0.241321,0.395459,0.177565,0.363995,0.684219,0.923054,0.409901,0.363995,...,0.493822,0.323077,0.082013,0.210101,0.363995,0.625107,0.241321,0.225,AKR,0


In [ ]:
# Get columns to normalize (excluding 'Company name', 'Period End Date', 'Fraud')
cols_to_normalize = [col for col in ratio_df.columns if col not in ['Company name', 'Period End Date', 'Fraud']]

# Normalize ratio_df
ratio_df_normalized = normalize_by_group(ratio_df, 'Company name', cols_to_normalize)

# Display the normalized DataFrame
ratio_df_normalized.head()

,Period End Date,Current Ratio,Pretax ROE,Times Interest Earned,Reinvestment Rate,Debt/Equity,Quick Ratio,Gross Margin,ROE,x Earnings Retention,...,(Total Debt - Cash) / EBITDA,x Pretax Margin,Effective Tax Rate,Pretax ROA,EBITDA Margin,Assets/Equity,Operating Margin,Asset Turnover,Company name,Fraud
0,Jun-2004,0.000000,0.385819,0.252199,0.460699,0.000000,0.000000,0.875502,0.423695,1.0,...,0.0,0.518499,0.010569,0.453901,0.415693,0.000000,0.500260,0.0,AKR,0
1,Dec-2004,0.311189,0.385819,0.000000,0.460699,0.472973,0.371622,0.620482,0.423695,1.0,...,0.0,0.420375,0.010569,0.453901,0.000000,0.545035,0.409870,0.0,AKR,0
2,Mar-2005,0.000000,0.385819,0.134897,0.460699,0.000000,0.000000,0.253012,0.423695,1.0,...,0.0,0.478820,0.016266,0.453901,0.188648,0.000000,0.466494,0.0,AKR,0
3,Jun-2005,0.496503,0.385819,0.321114,0.460699,0.169884,0.533784,0.534137,0.423695,1.0,...,0.0,0.558177,0.010344,0.453901,0.430718,0.275982,0.540260,0.0,AKR,0
4,Sep-2005,0.000000,0.385819,0.260997,0.460699,0.000000,0.000000,0.514056,0.423695,1.0,...,0.0,0.548525,0.010569,0.453901,0.404007,0.000000,0.532468,0.0,AKR,0


In [ ]:
# Separate features (X) and target (Y)
X_bs = bs_df_normalized.drop(columns=['Fraud', 'Period End Date','Company name'])  # All columns except 'Fraud'
Y_bs = bs_df_normalized['Fraud']  # Only the 'Fraud' column

# Display the shapes of X and Y to confirm the split
print("Shape of X:", X_bs.shape)
print("Shape of Y:", Y_bs.shape)

Shape of X: (851, 42)
Shape of Y: (851,)


### bs_df

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
# Typically, 80% for training and 20% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X_bs, Y_bs, test_size=0.2, random_state=42)

# Display the shapes of the datasets to confirm the split
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train: (680, 42)
Shape of X_test: (171, 42)
Shape of Y_train: (680,)
Shape of Y_test: (171,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. Create Logistic Regression model
logreg_model = LogisticRegression(random_state=42)

# 2. Train Logistic Regression model
logreg_model.fit(X_train, Y_train)

# 3. Create Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# 4. Train Random Forest model
rf_model.fit(X_train, Y_train)

# 5. Make predictions on the test set
logreg_predictions = logreg_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

# 6. Evaluate model performance
def evaluate_model(predictions, Y_test):
    accuracy = accuracy_score(Y_test, predictions)
    precision = precision_score(Y_test, predictions)
    recall = recall_score(Y_test, predictions)
    f1 = f1_score(Y_test, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

print("Logistic Regression Performance:")
evaluate_model(logreg_predictions, Y_test)

print("\nRandom Forest Performance:")
evaluate_model(rf_predictions, Y_test)

Logistic Regression Performance:
Accuracy: 0.9415
Precision: 0.9333
Recall: 0.6087
F1 Score: 0.7368

Random Forest Performance:
Accuracy: 0.9766
Precision: 1.0000
Recall: 0.8261
F1 Score: 0.9048
